In [ ]:
import random
from tqdm import tqdm
from MonBigTool import IS_levenshtein_distance_and_operations
from MonBigTool import levenshtein_distance_and_operations
from MonBigTool import MonBigTool,MASKmodel

mASKmodel = MASKmodel()
monBigTool = MonBigTool()
WordsDict = monBigTool.getWordsDict()


In [ ]:
MASK = monBigTool.getMASK()
WINDOW = 3

LOSS_PScore = 0
LOSS_PError = 0
LOSS_PMiss = 0
LOSS_PTrue = 0
LOSS_PTrue_errFix = 0
LOSS_PTrue_succFix = 0
LOSS_O_T = 0
LOSS_O_F = 0
LOSS_O_MISS = 0


COUNT = 0
LENMASK = len(MASK)
for ii in tqdm(range(LENMASK)):
    ii = random.choice(range(LENMASK))
    i = MASK[ii]
    COUNT += 1
    # i = random.choice()
    inode,tsen,fsen,sen,freq,pairs = monBigTool.Decode(i)

    # ########################################
    # # 得到词对 信息  ########################
    # for g in pairs:
    #     if not g[0].isalpha():
    #         continue


    #     # te = monBigTool.letterSim(g[0],g[1])
    #     # if te < 0.8:
    #         # print(g[0],g[1],te)
    # ########################################


    temp = mASKmodel.tomask(fsen,WINDOW,False)
    SecondaryTreatment = []
    for i in range(len(fsen)):
        if not fsen[i].isalpha():
            continue
        if fsen[i] in temp[i] :
            continue
        if monBigTool.mysterious(fsen[i]):
            continue

        if i in inode:
            LOSS_O_T += 1
            inode.remove(i)
        else:
            LOSS_O_F += 1
        SecondaryTreatment.append(i)
    LOSS_O_MISS += len(inode)
    # print('假----',' '.join(fsen),'----')
    # print('真----',' '.join(tsen),'----')
    # print('MASK----',' '.join(sen),'----')
    # print('怀疑----',SecondaryTreatment,'----',inode)


    # for i in SecondaryTreatment:
    #     Alternate = []
    #     candidate,score = monBigTool.FuzzySearch(fsen[i])
    #     Target = fsen[i]

    #     TTEMP = mASKmodel.hybridPrediction(fsen,i,3,candidate,score)
    #     # TTEMP = candidate
    #     print('目标',Target,':  候选',candidate)
    #     print('结果:',TTEMP)


    #     if len(TTEMP) == 0:
    #         continue

    #     if fsen[i] == TTEMP[0]:
    #         continue
    
    #     if i not in inode:
    #         LOSS_PError += 1
    #         print('*ERR'*20)
    #         print(Target,TTEMP)
    #     elif i in inode:
    #         LOSS_PTrue += 1
    #         inode.remove(i)
    #         if tsen[i] == TTEMP[0]:
    #             print('*SUCFix'*20)
    #             LOSS_PTrue_succFix += 1
    #         else:
    #             print('*ERRFix'*20)
    #             LOSS_PTrue_errFix += 1
    #         print(Target,TTEMP)
    
    # LOSS_PMiss += len(inode)
    # for i in inode:
    #     print('@MISS'*20)
    #     print('----',' '.join(fsen),'----')
    #     print('----',' '.join(tsen),'----')
    #     print('----',' '.join(sen),'----')
    #     print(fsen[i])
    #     print(fsen[i])

    if COUNT % 50 == 0:
    # print('-----------------')
        print('O_T:',LOSS_O_T)
        print('O_F:',LOSS_O_F)
        print('O_MISS:',LOSS_O_MISS)
    # print('PError:',LOSS_PError)
    # print('PMiss:',LOSS_PMiss)
    # print('PTrue:',LOSS_PTrue)
    # print('PTrue_succFix:',LOSS_PTrue_succFix)
    # print('PTrue_errFix:',LOSS_PTrue_errFix)
    


In [ ]:
# PError: 61
# PMiss: 37
# PTrue: 161
# PTrue_succFix: 140
# PTrue_errFix: 21
#   0%|          | 101/43061 [11:21<80:33:02,  6.75s/it]


# PError: 62
# PMiss: 27
# PTrue: 150
# PTrue_succFix: 135
# PTrue_errFix: 15
#   0%|          | 101/43061 [14:54<143:19:06, 12.01s/it]  添加词频权重



# PError: 67
# PMiss: 21
# PTrue: 146
# PTrue_succFix: 124
# PTrue_errFix: 22


In [ ]:
for ii in range(len(MASK)):
    i = MASK[ii]
    inode,tsen,fsen,sen,freq,pairs = monBigTool.Decode(i)
    if fsen[0] == 'энэ':
        print(ii)


In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained('tugstugi/bert-large-mongolian-uncased', use_fast=False)
model = AutoModelForMaskedLM.from_pretrained('tugstugi/bert-large-mongolian-uncased')

# 词嵌入
input_text = "хэрэв зориудаар алсан оол тэр хэмжээний малаар торгууль төлж цайруулмуй"
              
encoded_input = tokenizer.encode(input_text, return_tensors='pt')
outputs = model(encoded_input)


last_hidden_states = outputs[0]
# 解码 last_hidden_states

# 选择每个位置最可能的 token ID
predicted_ids = torch.argmax(last_hidden_states[0], dim=1)

print(input_text)
print(tokenizer.decode(predicted_ids.tolist()))


# 真---- хэрэв зориудаар алсан бол тэр хэмжээний малаар торгууль төлж цайруулмуй . ----
# MASK---- хэрэв зориудаар алсан [MASK] тэр хэмжээний малаар торгууль төлж [MASK] . ----
# 怀疑---- [3, 6, 9] ---- [3, 9]
# обл :  : ['сбл', 'ойл', 'обя', 'обг', 'убл', 'ол', 'оол', 'орл', 'оёл', 'об', 'бл', 'оби', 'оба', 'онл', 'нобл', 'обь', 'обх', 'дбл', 'ббл', 'обс', 'олл', 'оул', 'обт', 'обл', 'обе', 'эбл', 'обо', 'облж', 'шбл', 'обол', 'обу', 'осл', 'гобл', 'оил', 'общ', 'охл', 'хобл', 'обла', 'собл', 'обб', 'оел', 'объ', 'робл', 'добл', 'отл', 'нбл', 'лбл', 'обш', 'мбл']
# 结果: ['мбл', 'нбл', 'мол', 'бол', 'нол']

In [ ]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained('tugstugi/bert-large-mongolian-uncased', use_fast=True)
model = AutoModelForMaskedLM.from_pretrained('tugstugi/bert-large-mongolian-uncased',output_hidden_states=True)

# 词嵌入
input_text = "хэрэв зориудаар алсан оол тэр хэмжээний малаар торгууль төлж цайруулмуй"
              
encoded_input = tokenizer.encode(input_text, return_tensors='pt')
outputs = model(encoded_input)

outputs.logits.shape
len(outputs.hidden_states)




In [ ]:
from transformers import BertForTokenClassification, Trainer, TrainingArguments

# 加载预训练模型，指定要进行分类的类别数量
model = BertForTokenClassification.from_pretrained('tugstugi/bert-large-mongolian-uncased', num_labels=2)

# 定义训练参数
training_args = TrainingArguments(
    output_dir='./results',          # 输出目录
    num_train_epochs=3,              # 总的训练轮数
    per_device_train_batch_size=16,  # 每个设备的训练批次大小
    per_device_eval_batch_size=64,   # 每个设备的评估批次大小
    warmup_steps=500,                # 预热步数
    weight_decay=0.01,               # 权重衰减
    logging_dir='./logs',            # 日志目录
)

# 定义训练器
trainer = Trainer(
    model=model,                         # 要训练的模型
    args=training_args,                  # 训练参数
    train_dataset=train_dataset,         # 训练数据集
    eval_dataset=test_dataset            # 评估数据集
)

# 开始训练
trainer.train()

In [ ]:
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments,AutoTokenizer,AutoModelForMaskedLM
from torch.utils.data import Dataset


    # 读取


class SpellingErrorDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        labels = self.labels[idx]

        # 对文本进行编码
        encoding = self.tokenizer.encode_plus(
            text,
            truncation=True,
            max_length=self.max_len,
            padding='max_length',
            return_tensors='pt',
        )

        # 将标签对齐到编码的长度
        labels = labels + [0] * (self.max_len - len(labels))

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.long)
        }
tokenizer = AutoTokenizer.from_pretrained('tugstugi/bert-large-mongolian-uncased', use_fast=False)
model = AutoModelForMaskedLM.from_pretrained('tugstugi/bert-large-mongolian-uncased')
# 创建分词器

# 创建数据集
texts = ["зэрэг, хамгаалагчид!", "Goodbye, world!"]
labels = [[0, 0, 1, 0], [0, 0, 0, 1]]  # 假设我们有两个类别，0表示正确，1表示拼写错误
dataset = SpellingErrorDataset(texts, labels, tokenizer, max_len=16)

In [ ]:
# dataset 打印
print(dataset[0])

In [ ]:
OUT = []
for i in tqdm(monBigTool.getMASK()):
    sen = []
    labels = []
    for ss in i['sen']:
        if ss == '<>':
            i['word'].pop(0)
            sen.append(i['word'].pop(0))
            count = sen[-1]
            count = len(tokenizer(count)['input_ids'])
            labels += [1] * count
        else:
            sen.append(ss)
            count = sen[-1]
            count = len(tokenizer(count)['input_ids'])
            labels += [0] * count
    OUT.append({
        'sen':sen,
        'labels':labels
    })
# 保存OUT
import json
with open('sentencePair.json','w') as f:
    json.dump(OUT,f)

# 读取
import json
with open('sentencePair.json','r') as f:
    OUT = json.load(f)
    # 读取
import json
with open('sentencePair.json','r') as f:
    OUT = json.load(f)

In [13]:
from transformers import BertForTokenClassification, AutoTokenizer
MODELNAME = 'tugstugi/bert-large-mongolian-uncased'
# 加载模型和分词器
model = BertForTokenClassification.from_pretrained('./RRRRRRRRRRR')
tokenizer = AutoTokenizer.from_pretrained(MODELNAME, use_fast=False)

# 输入你的句子
sentence = "цаашлаад бид чуулганыхаа библийн сургалт , илгээлт гээд юу эсэхийг ярихсан билээ ."


# 对句子进行编码
inputs = tokenizer(sentence, return_tensors="pt")

# 获取模型的预测结果
outputs = model(**inputs)

# 获取预测的标签
predictions = outputs.logits.argmax(-1).tolist()

print(predictions)



tokenizer = AutoTokenizer.from_pretrained(MODELNAME, use_fast=False)


sen = sentence.split()
Tindex = 0
for j in range(len(sen)):
    aAaaa = tokenizer.tokenize(sen[j])
    print(sen[j],len(aAaaa),predictions[0][Tindex:Tindex+len(aAaaa)])
    Tindex = Tindex + len(aAaaa)

Some weights of the model checkpoint at ./RRRRRRRRRRR were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[[0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]]
цаашлаад 1 [0]
бид 1 [0]
чуулганыхаа 2 [0, 0]
библийн 3 [0, 1, 1]
сургалт 1 [1]
, 2 [0, 0]
илгээлт 1 [0]
гээд 1 [0]
юу 1 [0]
эсэхийг 1 [0]
ярихсан 2 [0, 1]
билээ 1 [0]
. 2 [0, 0]
